<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/RAG-gsm8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install tiktoken
!pip install langchain-openai
!pip install faiss-cpu
!pip install langchain-community
!pip install sentence_transformers
!pip install peft
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/32

In [ ]:
import os
from pathlib import Path
import subprocess
import shutil
from langchain.text_splitter import TokenTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import pandas as pd
import openai
import faiss
import numpy as np

from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModel, PeftModel
from datasets import load_dataset

import pickle

In [ ]:
# 모델 경로
model_path = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
model = SentenceTransformer(model_path, device=device)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
# Load csv file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

dataset = load_dataset("koutch/staqc", 'sca_python')
df = dataset['train'].to_pandas()
df

,question_id,question,snippet
0,18601725,Using Regex (or simmilar) to change song name ...,set ASTID to AppleScript's text item delimiter...
1,21889025,Product of two beta distributions,set.seed(1) # for reproducability\n\nn <- 1000...
2,26661808,how to grab from JSON in selenium python,from bs4 import BeautifulSoup\nimport json\n\n...
3,7523511,Unable to use wx.NotificationMessage properly ...,"import wx, sys\n\ntry:\n import win32gui #,..."
4,34996225,Django: Add additional properties to User,if request.method == 'POST':\n new_user = U...
...,...,...,...
85289,18219007,How do I write a YAML file using a dictionary ...,"yaml.dump(d, width=20, default_flow_style=Fal..."
85290,4325372,using list with python multiprocessing,"import time, random\nfrom multiprocessing impo..."
85291,10787103,2d hsv color space in matplotlib,import numpy as np\nimport pylab as pl\nfrom m...
85292,7602174,How to only print lines with unique fields?,sort input-file.txt | uniq -u -w 3\n


In [ ]:
embeddings = []

for i, question in enumerate(df['question'].tolist()):
    embedding = model.encode(question)
    embeddings.append(embedding)

def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

# 임베딩을 Numpy 배열로 변환
embedding_matrix = np.array(embeddings).astype('float32')

# 정규화
embedding_matrix_normalized = normalize(embedding_matrix)

# FAISS 인덱스 생성 및 벡터 추가
index = faiss.IndexFlatIP(embedding_matrix_normalized.shape[1])
index.add(embedding_matrix_normalized)

# faiss 인덱스 저장
faiss.write_index(index, 'question_faiss_index-sentence.faiss')

In [ ]:
!cp question_faiss_index-sentence.faiss /content/drive/MyDrive/

In [ ]:
print(f"Number of questions: {len(df)}")
print(f"Number of embeddings: {len(embeddings)}")

Number of questions: 85294
Number of embeddings: 85294


In [ ]:
## 여기서부터는 인덱스 로드해서 사용하는 것. FAISS 인덱스 로드

index = faiss.read_index('question_faiss_index-sentence.faiss')

In [ ]:
# 유사도 검색 함수
def search_similar_questions(query_sentence, k=10):
    query_embedding = model.encode([query_sentence])
    query_embedding_normalized = normalize(query_embedding)

    # FAISS 인덱스에서 가장 유사한 벡터 k개 검색
    distances, indices = index.search(np.array([query_embedding_normalized[0]], dtype='float32'), k)

    # 검색된 인덱스와 유사도를 반환
    similar_questions = []
    for i in range(k):
        question = df['question'][indices[0][i]]
        snippet = df['snippet'][indices[0][i]]
        # plan = df['plan'][indices[0][i]]

        print(f"문서 id: {indices[0][i]}, 유사도: {distances[0][i]}, 질문: {question}")
        similar_questions.append((question, snippet))

    return similar_questions

In [ ]:
## FAISS 인덱스에서 유사한 벡터 검색
query_sentence = "Python file modes detail ?"

similar_questions = search_similar_questions(query_sentence, k=1)
print(similar_questions[0][0], similar_questions[0][1])

문서 id: 8, 유사도: 0.9847220778465271, 질문: Python file modes detail
Python file modes detail s = mode;
while (*s) {
    switch (*s++) {
    case 'r':
        if (rwa) {
        bad_mode:
            PyErr_SetString(PyExc_ValueError,
                    "Must have exactly one of read/write/append mode");
            goto error;
        }
        rwa = 1;
        self->readable = 1;
        break;
    case 'w':
        if (rwa)
            goto bad_mode;
        rwa = 1;
        self->writable = 1;
        flags |= O_CREAT | O_TRUNC;
        break;
    case 'a':
        if (rwa)
            goto bad_mode;
        rwa = 1;
        self->writable = 1;
        flags |= O_CREAT;
        append = 1;
        break;
    case 'b':
        break;
    case '+':
        if (plus)
            goto bad_mode;
        self->readable = self->writable = 1;
        plus = 1;
        break;
    default:
        PyErr_Format(PyExc_ValueError,
                 "invalid mode: %.200s", mode);
        goto error;
 

In [ ]:
# from google.colab import userdata
# from ftplib import FTP

# ftp_server = "121.136.96.223"
# ftp_port = 21
# ftp_username = "donghuna_ftp"
# ftp_password = userdata.get('FTP_PASSWD')
# folder_path = "database/PromptGenerate/"

# ftp = FTP()
# ftp.connect(ftp_server, ftp_port)
# ftp.login(ftp_username, ftp_password)
# ftp.set_pasv(True)

In [ ]:
# # ftp 경로에 파일 저장
# import io

# csv_buffer = io.StringIO()
# df.to_csv(csv_buffer, index=True)

# csv_buffer.seek(0) # buffer의 시작으로 이동
# ftp.storbinary(f"STOR {folder_path}test.csv", io.BytesIO(csv_buffer.getvalue().encode()))